In [ ]:
!python --version

Python 3.7.13


In [ ]:
!sudo apt-get install -y build-essential libpython3.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libpython3.7 is already the newest version (3.7.13-1+bionic3).
libpython3.7 set to manually installed.
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
!pip install --upgrade pip
!pip install --upgrade setuptools
!pip install recommenders

     |████████████████████████████████| 2.1 MB 12.8 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.6/335.6 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install papermill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.5/428.5 KB 29.6 MB/s eta 0:00:00
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.1 which is incompatible.


In [ ]:
!pip install scrapbook

In [ ]:
!pip install xlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 42.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for xlearn: filename=xlearn-0.40a1-py3-none-any.whl size=229988 sha256=d6d31b3af04565b4f9b544633f3de7ab5aab47ae62c5347cc32528ba71488318
  Stored in directory: /root/.cache/pip/wheels/1e/6d/eb/256faaf8142a9363431e2dc3f6a3d1c9080f526a1e5475049d
Successfully built xlearn


# FFM - xlearn crushes

In [ ]:
import sys
import os
import papermill as pm
import scrapbook as sb
from tempfile import TemporaryDirectory
import xlearn as xl
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib notebook
from matplotlib import pyplot as plt

from recommenders.utils.constants import SEED
from recommenders.utils.timer import Timer
from recommenders.datasets.download_utils import maybe_download, unzip_file
from recommenders.tuning.parameter_sweep import generate_param_grid
from recommenders.datasets.pandas_df_utils import LibffmConverter

print("System version: {}".format(sys.version))
print("Xlearn version: {}".format(xl.__version__))

System version: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
Xlearn version: 0.4.0


In [ ]:
# df_feature_original = pd.DataFrame({
#     'rating': [1, 0, 0, 1, 1],
#     'field1': ['xxx1', 'xxx2', 'xxx4', 'xxx4', 'xxx4'],
#     'field2': [3, 4, 5, 6, 7],
#     'field3': [1.0, 2.0, 3.0, 4.0, 5.0],
#     'field4': ['1', '2', '3', '4', '5']
# })

# converter = LibffmConverter().fit(df_feature_original, col_rating='rating')
# df_out = converter.transform(df_feature_original)
# df_out

In [ ]:
# Model parameters
LEARNING_RATE = 0.2
LAMBDA = 0.002
EPOCH = 10
OPT_METHOD = "sgd" # options are "sgd", "adagrad" and "ftrl"

# The metrics for binary classification options are "acc", "prec", "f1" and "auc"
# for regression, options are "rmse", "mae", "mape"
METRIC = "auc" 

In [ ]:
# Paths
YAML_FILE_NAME = "xDeepFM.yaml"
TRAIN_FILE_NAME = "cretio_tiny_train"
VALID_FILE_NAME = "cretio_tiny_valid"
TEST_FILE_NAME = "cretio_tiny_test"
MODEL_FILE_NAME = "model.out"
OUTPUT_FILE_NAME = "output.txt"

tmpdir = TemporaryDirectory()

data_path = tmpdir.name
yaml_file = os.path.join(data_path, YAML_FILE_NAME)
train_file = os.path.join(data_path, TRAIN_FILE_NAME)
valid_file = os.path.join(data_path, VALID_FILE_NAME)
test_file = os.path.join(data_path, TEST_FILE_NAME)
model_file = os.path.join(data_path, MODEL_FILE_NAME)
output_file = os.path.join(data_path, OUTPUT_FILE_NAME)

assets_url = "https://recodatasets.z20.web.core.windows.net/deeprec/xdeepfmresources.zip"
assets_file = maybe_download(assets_url, work_directory=data_path)
unzip_file(assets_file, data_path)

100%|██████████| 10.3k/10.3k [00:01<00:00, 6.84kKB/s]


In [ ]:
# Training task
ffm_model = xl.create_ffm()        # Use field-aware factorization machine (ffm)
ffm_model.setTrain(train_file)     # Set the path of training dataset
ffm_model.setValidate(valid_file)  # Set the path of validation dataset

# Parameters:
#  0. task: binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: auc
#  4. number of epochs: 10
#  5. optimization method: sgd
param = {"task":"binary", 
         "lr": LEARNING_RATE, 
         "lambda": LAMBDA, 
         "metric": METRIC,
         "epoch": EPOCH,
         "opt": OPT_METHOD
        }

In [ ]:
# Start to train
# The trained model will be stored in model.out
with Timer() as time_train:
    ffm_model.fit(param, model_file)
print(f"Training time: {time_train}")

In [ ]:
!dir

sample_data


In [ ]:
!dir ..

bin	 datalab  home	 lib64	opt	    root  srv		     tmp    var
boot	 dev	  lib	 media	proc	    run   sys		     tools
content  etc	  lib32  mnt	python-apt  sbin  tensorflow-1.15.2  usr


In [ ]:
train_file

'/tmp/tmpeyjtcjdx/cretio_tiny_train'

In [ ]:
!cat /tmp/tmpeyjtcjdx/sample_FFM_data.txt

0.000 1:356:1 2:817:1 3:1455:1 4:1822:1 5:2040:1 6:3078:1 7:4134:1 8:4970:1 9:5148:1 10:5161:1 11:5245:1 12:5668:1 13:5710:1 14:6430:1 15:7334:1 16:7390:1 17:7567:1 18:8330:1 19:8788:1 20:9910:1
0.000 1:179:1 2:649:1 3:1660:1 4:1828:1 5:2385:1 6:2876:1 7:3473:1 8:4949:1 9:5108:1 10:5180:1 11:5388:1 12:5626:1 13:5839:1 14:6435:1 15:7190:1 16:7340:1 17:7783:1 18:8490:1 19:8810:1 20:9751:1
0.000 1:202:1 2:554:1 3:1047:1 4:1827:1 5:2191:1 6:3147:1 7:3813:1 8:4963:1 9:5049:1 10:5189:1 11:5263:1 12:5642:1 13:5908:1 14:6595:1 15:7114:1 16:7382:1 17:7825:1 18:8665:1 19:8700:1 20:9787:1
0.000 1:11:1 2:821:1 3:1285:1 4:1805:1 5:2263:1 6:3320:1 7:4518:1 8:4967:1 9:5097:1 10:5173:1 11:5504:1 12:5588:1 13:5825:1 14:6396:1 15:7095:1 16:7510:1 17:8256:1 18:8338:1 19:8706:1 20:9574:1
0.000 1:454:1 2:823:1 3:1422:1 4:1821:1 5:2257:1 6:2621:1 7:4747:1 8:4953:1 9:5067:1 10:5176:1 11:5232:1 12:5592:1 13:5674:1 14:6391:1 15:6904:1 16:7486:1 17:7698:1 18:8392:1 19:8726:1 20:8878:1
0.000 1:165:1 2:679:1 3:16

# LightFM

In [ ]:
import sys
import os

import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation

# Import LightFM's evaluation metrics
from lightfm.evaluation import precision_at_k as lightfm_prec_at_k
from lightfm.evaluation import recall_at_k as lightfm_recall_at_k

# Import repo's evaluation metrics
from recommenders.evaluation.python_evaluation import (
    precision_at_k, recall_at_k)

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.models.lightfm.lightfm_utils import (
    track_model_metrics, prepare_test_df, prepare_all_predictions,
    compare_metric, similar_users, similar_items)

print("System version: {}".format(sys.version))
print("LightFM version: {}".format(lightfm.__version__))

System version: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
LightFM version: 1.16


In [ ]:
# Select MovieLens data size
MOVIELENS_DATA_SIZE = '100k'

# default number of recommendations
K = 10
# percentage of data used for testing
TEST_PERCENTAGE = 0.25
# model learning rate
LEARNING_RATE = 0.25
# no of latent factors
NO_COMPONENTS = 20
# no of epochs to fit model
NO_EPOCHS = 20
# no of threads to fit model
NO_THREADS = 4
# regularisation for both user and item features
ITEM_ALPHA=1e-6
USER_ALPHA=1e-6

# seed for pseudonumber generations
SEEDNO = 42

In [ ]:
data = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    genres_col='genre',
    year_col='Year',
    header=["userID", "itemID", "rating"]
)
# quick look at the data
data.sample(5)

100%|██████████| 4.81k/4.81k [00:00<00:00, 13.6kKB/s]


,userID,itemID,rating,genre,Year
21022,675,272,3.0,Drama,1997
41150,477,709,5.0,Comedy|Romance,1992
19270,887,385,4.0,Action|Adventure|Comedy|Romance,1994
71428,539,59,5.0,Drama,1994
51935,415,185,4.0,Horror|Romance|Thriller,1960


In [ ]:
# split the genre based on the separator
movie_genre = [x.split('|') for x in data['genre']]

In [ ]:
# retrieve the all the unique genres in the data
all_movie_genre = sorted(list(set(itertools.chain.from_iterable(movie_genre))))
# quick look at the all the genres within the data
all_movie_genre

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western',
 'unknown']

In [ ]:
user_feature_URL = 'https://files.grouplens.org/datasets/movielens/ml-100k/u.user'
user_data = pd.read_table(user_feature_URL, 
              sep='|', header=None)
user_data.columns = ['userID','age','gender','occupation','zipcode']

# merging user feature with existing data
new_data = data.merge(user_data[['userID','occupation']], left_on='userID', right_on='userID')
# quick look at the merged data
new_data.head(5)

,userID,itemID,rating,genre,Year,occupation
0,196,242,3.0,Comedy,1996,writer
1,196,257,2.0,Action|Adventure|Comedy|Sci-Fi,1997,writer
2,196,111,4.0,Comedy|Romance,1996,writer
3,196,25,4.0,Comedy,1996,writer
4,196,382,4.0,Comedy|Drama,1994,writer


In [ ]:
all_occupations = sorted(list(set(new_data['occupation'])))
all_occupations

['administrator',
 'artist',
 'doctor',
 'educator',
 'engineer',
 'entertainment',
 'executive',
 'healthcare',
 'homemaker',
 'lawyer',
 'librarian',
 'marketing',
 'none',
 'other',
 'programmer',
 'retired',
 'salesman',
 'scientist',
 'student',
 'technician',
 'writer']

In [ ]:
for genre in all_movie_genre:
    new_data[genre] = new_data['genre'].str.contains(genre)*1
new_data.head(5)

,userID,itemID,rating,genre,Year,occupation,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,unknown
0,196,242,3.0,Comedy,1996,writer,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,196,257,2.0,Action|Adventure|Comedy|Sci-Fi,1997,writer,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,196,111,4.0,Comedy|Romance,1996,writer,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,196,25,4.0,Comedy,1996,writer,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,196,382,4.0,Comedy|Drama,1994,writer,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dataset2 = Dataset()
dataset2.fit(data['userID'], 
            data['itemID'], 
            item_features=all_movie_genre,
            user_features=all_occupations)

In [ ]:
item_features = dataset2.build_item_features(
    (x, y) for x,y in zip(data.itemID, movie_genre))

In [ ]:
user_features = dataset2.build_user_features(
    (x, [y]) for x,y in zip(new_data.userID, new_data['occupation']))

In [ ]:
(interactions2, weights2) = dataset2.build_interactions(data.iloc[:, 0:3].values)

train_interactions2, test_interactions2 = cross_validation.random_train_test_split(
    interactions2, test_percentage=TEST_PERCENTAGE,
    random_state=np.random.RandomState(SEEDNO))

In [ ]:
model2 = LightFM(loss='warp', no_components=NO_COMPONENTS, 
                 learning_rate=LEARNING_RATE, 
                 item_alpha=ITEM_ALPHA,
                 user_alpha=USER_ALPHA,
                 random_state=np.random.RandomState(SEEDNO))

In [ ]:
model2.fit(interactions=train_interactions2,
           user_features=user_features,
           item_features=item_features,
           epochs=NO_EPOCHS);

In [ ]:
_, user_embeddings = model2.get_user_representations(features=user_features)
user_embeddings

array([[ 2.7921509e-03, -3.2446080e-01,  1.4887877e+00, ...,
         2.9964762e+00, -2.8919451e+00,  3.8470132e+00],
       [-3.5523415e-01,  6.4026421e-01,  2.4279106e+00, ...,
         3.5219254e+00, -4.1111979e+00,  3.8146427e+00],
       [ 1.4626522e-01,  1.2756311e+00,  1.3573985e+00, ...,
         1.8104703e+00, -5.0653615e+00,  4.3718629e+00],
       ...,
       [ 1.2883155e+00, -7.3635006e-01,  2.2852180e+00, ...,
         2.2419245e+00, -2.5155954e+00,  3.2238843e+00],
       [-3.4913281e-01,  6.2562793e-01,  2.3744226e+00, ...,
         3.4396946e+00, -3.9925151e+00,  3.6938140e+00],
       [-4.0814415e-01,  1.0403929e+00,  1.8536233e+00, ...,
         3.1999631e+00, -3.9003000e+00,  4.5680871e+00]], dtype=float32)

In [ ]:
# Cosine similarity
similar_users(user_id=1, user_features=user_features, 
            model=model2)

,userID,score
0,54,1.000000
1,555,1.000000
2,314,0.999999
3,395,0.999998
4,465,0.999998
5,481,0.999998
6,282,0.999997
7,527,0.999997
8,411,0.999997
9,855,0.999994


In [ ]:
_, item_embeddings = model2.get_item_representations(features=item_features)
item_embeddings

array([[ 0.26047313,  0.3036962 ,  0.4571161 , ...,  0.5463799 ,
        -0.78396976,  0.7428898 ],
       [ 0.1876924 ,  0.2716511 ,  0.5517744 , ...,  0.7183491 ,
        -0.6560396 ,  0.7236648 ],
       [ 0.12680474,  0.39027125,  0.48408258, ...,  0.53494716,
        -0.7423564 ,  0.61817217],
       ...,
       [ 0.29515266,  0.02557752,  0.29958707, ...,  0.04983175,
        -0.12842962,  0.04753447],
       [-0.44294164, -0.32489085,  0.44517514, ...,  0.89324677,
        -0.28879496,  0.6795278 ],
       [-0.19142139, -0.3599534 ,  0.67460454, ...,  0.60447884,
        -0.52155465,  0.63757414]], dtype=float32)

In [ ]:
# Cosine similarity
similar_items(item_id=10, item_features=item_features, 
            model=model2)

,itemID,score
0,231,0.998594
1,12,0.998594
2,417,0.998593
3,182,0.998573
4,474,0.998553
5,76,0.998511
6,253,0.998499
7,296,0.998407
8,573,0.998405
9,458,0.998372


## Simple k-means

In [ ]:
from sklearn.cluster import KMeans

### Movie genre

In [ ]:
print("Proportion of movies of specific genres for all data:")
mean_genre = new_data[all_movie_genre].mean()
mean_genre.sort_values(ascending=False)

Proportion of movies of specific genres for all data:


Drama          0.39895
Comedy         0.29832
Action         0.25589
Thriller       0.21872
Romance        0.19461
Adventure      0.13753
Sci-Fi         0.12730
War            0.09398
Crime          0.08055
Children's     0.07182
Horror         0.05317
Mystery        0.05245
Musical        0.04954
Animation      0.03605
Western        0.01854
Film-Noir      0.01733
Fantasy        0.01352
Documentary    0.00758
unknown        0.00010
dtype: float64

In [ ]:
# n_clusters = len(all_movie_genre)
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(user_embeddings)

In [ ]:
top_k = 5
for i in range(n_clusters):
    cluster_ids = np.argwhere(kmeans.labels_ == i).ravel()
    cluster_df = new_data[new_data["userID"].isin(cluster_ids)]
    print(f"Proportion of movies of top {top_k} genres for cluster {i}:")
    print((cluster_df[all_movie_genre].mean()/mean_genre).sort_values(ascending=False)[:top_k])
    print()

Proportion of movies of top 5 genres for cluster 0:
Western        1.154988
Film-Noir      1.137522
Documentary    1.024516
War            1.024353
Comedy         1.011855
dtype: float64

Proportion of movies of top 5 genres for cluster 1:
unknown       2.159711
Animation     1.106814
Children's    1.106619
Fantasy       1.070271
Horror        1.041877
dtype: float64

Proportion of movies of top 5 genres for cluster 2:
Documentary    1.055649
Western        1.027496
Mystery        1.024585
Fantasy        1.022287
Thriller       1.011278
dtype: float64

Proportion of movies of top 5 genres for cluster 3:
unknown      1.090552
War          1.031217
Animation    1.021479
Drama        1.017976
Romance      1.014098
dtype: float64

Proportion of movies of top 5 genres for cluster 4:
Documentary    1.287084
Horror         1.177698
Mystery        1.110351
Musical        1.083136
Drama          1.034323
dtype: float64



In [ ]:
# n_clusters = len(all_movie_genre)
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(item_embeddings)

In [ ]:
top_k = 5
for i in range(n_clusters):
    cluster_ids = np.argwhere(kmeans.labels_ == i).ravel()
    cluster_df = new_data[new_data["itemID"].isin(cluster_ids)]
    cluster_df = cluster_df.drop_duplicates(subset=["itemID"])
    print(f"Proportion of movies of top {top_k} genres for cluster {i}:")
    print((cluster_df[all_movie_genre].mean()/mean_genre).sort_values(ascending=False)[:top_k])
    print()

Proportion of movies of top 5 genres for cluster 0:
Film-Noir    2.024681
Drama        1.275277
Musical      1.062406
Comedy       1.058560
Animation    0.973307
dtype: float64

Proportion of movies of top 5 genres for cluster 1:
unknown        23.781213
Documentary     5.019781
Film-Noir       1.097806
Horror          1.073442
Drama           1.061049
dtype: float64

Proportion of movies of top 5 genres for cluster 2:
Documentary    4.361194
Drama          1.242932
Comedy         0.997321
Western        0.891528
Crime          0.820803
dtype: float64

Proportion of movies of top 5 genres for cluster 3:
Documentary    2.766193
Western        1.217942
Horror         1.213393
Children's     1.122879
Comedy         1.075918
dtype: float64

Proportion of movies of top 5 genres for cluster 4:
Documentary    3.141098
Film-Noir      1.373891
Drama          1.133929
Comedy         1.037556
Children's     0.994550
dtype: float64



### User occupation

In [ ]:
print("Proportion of occupations of users for all data:")
occupation_counts = new_data["occupation"].value_counts()
occupation_counts.sort_values(ascending=False) / new_data.shape[0]

Proportion of occupations of users for all data:


student          0.21957
other            0.10663
educator         0.09442
engineer         0.08175
programmer       0.07801
administrator    0.07479
writer           0.05536
librarian        0.05273
technician       0.03506
executive        0.03403
healthcare       0.02804
artist           0.02308
entertainment    0.02095
scientist        0.02058
marketing        0.01950
retired          0.01609
lawyer           0.01345
none             0.00901
salesman         0.00856
doctor           0.00540
homemaker        0.00299
Name: occupation, dtype: float64

In [ ]:
# n_clusters = len(all_occupations)
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(user_embeddings)

In [ ]:
top_k = 5
for i in range(n_clusters):
    cluster_ids = np.argwhere(kmeans.labels_ == i).ravel()
    cluster_df = new_data[new_data["userID"].isin(cluster_ids)]
    cluster_df = cluster_df.drop_duplicates(subset=["userID"])
    print(f"Proportion of top {top_k} occupations of users for cluster {i}:")
    print(((cluster_df["occupation"].value_counts()/occupation_counts).sort_values(ascending=False))[:top_k])
    print()

Proportion of top 5 occupations of users for cluster 0:
homemaker    0.006689
salesman     0.005841
doctor       0.005556
marketing    0.003590
none         0.003330
Name: occupation, dtype: float64

Proportion of top 5 occupations of users for cluster 1:
none         0.004440
doctor       0.003704
marketing    0.003590
homemaker    0.003344
lawyer       0.002974
Name: occupation, dtype: float64

Proportion of top 5 occupations of users for cluster 2:
scientist        0.005345
salesman         0.004673
doctor           0.003704
homemaker        0.003344
entertainment    0.002864
Name: occupation, dtype: float64

Proportion of top 5 occupations of users for cluster 3:
homemaker    0.010033
artist       0.004333
scientist    0.003887
retired      0.003729
other        0.003282
Name: occupation, dtype: float64

Proportion of top 5 occupations of users for cluster 4:
marketing        0.002051
librarian        0.001517
scientist        0.001458
entertainment    0.001432
artist           0.0

In [ ]:
# n_clusters = len(all_movie_genre)
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(item_embeddings)

In [ ]:
top_k = 5
for i in range(n_clusters):
    cluster_ids = np.argwhere(kmeans.labels_ == i).ravel()
    cluster_df = new_data[new_data["itemID"].isin(cluster_ids)]
    print(f"Proportion of top {top_k} occupations of users for cluster {i}:")
    print(((cluster_df["occupation"].value_counts()/occupation_counts).sort_values(ascending=False))[:top_k])
    print()

Proportion of top 5 occupations of users for cluster 0:
healthcare    0.009629
retired       0.005594
artist        0.005199
librarian     0.003793
executive     0.002939
Name: occupation, dtype: float64

Proportion of top 5 occupations of users for cluster 1:
marketing    0.559487
retired      0.546302
writer       0.544798
artist       0.543761
educator     0.542470
Name: occupation, dtype: float64

Proportion of top 5 occupations of users for cluster 2:
healthcare    0.039586
homemaker     0.030100
none          0.027747
artist        0.027296
librarian     0.025033
Name: occupation, dtype: float64

Proportion of top 5 occupations of users for cluster 3:
doctor           0.446296
technician       0.445237
entertainment    0.444391
scientist        0.444121
student          0.443549
Name: occupation, dtype: float64

Proportion of top 5 occupations of users for cluster 4:
healthcare       0.011412
none             0.011099
executive        0.010285
entertainment    0.007637
homemaker 